In [1]:
import numpy as np  
from itertools import combinations
import pandas as pd

# CARGA DE DATOS

In [2]:
spotify = np.load("spotify.npy",allow_pickle=True)
spotify = spotify.tolist()

# FUNCIONES ADICIONALES

In [14]:
## 
def frecuency_each_song( spotify ):
    frecuencias = {}

    for i in range( len(spotify) ):
        for j in range(len(spotify[i])):
            song = spotify[i][j]
            if song in frecuencias: frecuencias[song] += 1
            else: frecuencias[song] = 1
    return frecuencias

def frecuency_each_combination( spotify, combination, n ):
    # RECORRER ARREGLO DE PLAYLISTS
    for i in range( len(spotify) ):   #len(arr)
        # RECORRER DICCIONARIO DE PARES
        for key, value in combination.items():
            playlist = []
            for j in range(n): playlist.append(key[j])
            # VERIFICAR SI COMBINACION SE ENCUENTRA EN PLAYLIST
            if( set(playlist).issubset(set( spotify[i] ))):  
                combination[key] += 1

    return combination

def create_combinations( array, n ):
    temp = list(combinations( array, n ))
    dic_combinations = {}
    for i in range(len(temp)): dic_combinations[ temp[i] ] = 0

    return dic_combinations

def pass_min_support( dic, min_support ):
    min_frecuency = []
    for key, value in dic.items():

        if (value / 10000) >= min_support:
            min_frecuency.append( key )
            
    return min_frecuency

def combinations_to_next_level( min_frecuencias2, nivel ):

    # RECUPERAMOS LOS N-1 ELEMENTOS DE CADA COMBINACION
    aux_primeros=[]
    
    for i in min_frecuencias2: aux_primeros.append( i[0] )

    # REALIZACMOS UN CONTEO PARA TRABAJAR CON AQUELLAS COMBINACIONES > 1
    aUnique = np.array(aux_primeros)
    unique, counts = np.unique(aUnique, return_counts=True)
    auxi = dict(zip(unique, counts))

    print(auxi)

    aux_primeros_fijos = []

    combinatorias = {}
    for key, value in auxi.items(): # CONTEO DE APARICIONES DE LOS N-1 ELEMENTOS
        
        adicionales = []
        if value > 1:
            aux_primeros_fijos.append( key )
            for auxiliar in min_frecuencias2: # BUSQUEDA DE ADICIONALES EN DICCIONARIO 
                if key == auxiliar[0]:
                    adicionales.append(auxiliar[1])

            adicionales = [key] + adicionales
            combinaciones = list(combinations(adicionales, nivel + 1))

            for i in combinaciones:  
                if i[0] in aux_primeros_fijos:
                    combinatorias[ i ] = 0

    return combinatorias

def combinations_rules(items):
    dic = []

    for i in range(len(items)):

        tamanio = len(items[i])
        adicionales = list( items[i] )

        combinaciones = list(combinations(adicionales, tamanio - 1))

        for combinacion in combinaciones:
            inicio = set(items[i]) - set(combinacion)
            fin = combinacion

            if [tuple(inicio), tuple(fin) ] not in dic:
                dic.append([tuple(inicio), tuple(fin) ])

            if [tuple(fin), tuple(inicio) ] not in dic:
                dic.append([tuple(fin), tuple(inicio) ])
    return dic

# FUNCION GET_FREQUENT_ITEMSETS

In [15]:
def get_frequent_itemsets(playlists, min_support):

    # FRECUENCIA DE APARICION DE CADA CANCION
    frecuency_of_songs = frecuency_each_song(playlists)

    # FILTRAR AQUELLAS CANCIONES QUE PASEN EL SOPORTE MINIMO
    songs_pass_support = pass_min_support( frecuency_of_songs, min_support )
    
    itemsets = []

    bandera = True
    nivel = 1
    combinaciones = {}

    while ( bandera ):
        nivel += 1

        # HALLA LAS COMBINACIONES PARA EL RESPECTIVO NIVEL
        if nivel == 2: combinaciones = create_combinations( songs_pass_support, nivel )

        # HALLA LA FRECUENCIA CON LA QUE APARECE CADA COMBINACION EN CADA PLAYLIST
        frecuencia_combinaciones = frecuency_each_combination( spotify, combinaciones, nivel )

        # FILTRAR AQUELLAS COMBINACIONES QUE PASEN EL SOPORTE MINIMO
        combinaciones_pass_support = pass_min_support( frecuencia_combinaciones, min_support )

        # AGREGAR COMBINACIONES A 'ITEMSETS'
        for combinacion in combinaciones_pass_support: itemsets.append( combinacion )

        # ENCONTRAR LAS NUEVAS COMBINACIONES PARA EL SIGUIENTE NIVEL
        combinaciones = combinations_to_next_level( combinaciones_pass_support, nivel )

        # print(combinaciones_pass_support)

        # SI EL TAMAÑO DE COMBINACIONES QUE PASEN EL SOPORTE MINIMO ES '1', CORTAMOS EL ALGORITMO
        if (len( combinaciones_pass_support ) <= 1) : bandera = False


    print( itemsets )

    return itemsets

# FUNCION GENERATE_ASSOCIATION_RULES

In [37]:
def generate_association_rules(frequent_itemsets, confidence = 0, lift = 0):

    # ENCONTRAMOS LAS REGLAS DE ASOCIACION, PARA LOS ITEMSETS QUE PASEN EL UMBRAL
    rules = combinations_rules(frequent_itemsets)

    count_x_y = [0] * len(rules)
    count_x   = [0] * len(rules)
    count_y   = [0] * len(rules)

    soporte   = [0] * len(rules)
    confianza = [0] * len(rules)
    sustento  = [0] * len(rules)

    for i in range( len(spotify) ):   #len(arr)
    # RECORRER DICCIONARIO DE PARES
        for j in range(len(rules)):
            
            x_y = list( rules[j][0] + rules[j][1] )
            x = list( rules[j][0] )
            y = list( rules[j][1] )

            # VERIFICAR SI COMBINACION SE ENCUENTRA EN PLAYLIST
            if( set(x_y).issubset(set( spotify[i] ))):  
                count_x_y[j] += 1
            if( set(x).issubset(set( spotify[i] ))):  
                count_x[j] += 1
            if( set(y).issubset(set( spotify[i] ))):  
                count_y[j] += 1

    for j in range(len(rules)):
        soporte[j] = (count_x_y[j] / len(spotify))
        confianza[j] = (count_x_y[j] / count_x[j])
        sustento[j] = confianza[j] / (count_y[j] / len(spotify))
    
    for j in range(len(rules)):
        if (confianza[j] >= confidence and sustento[j] >= lift):
            print(j+1)
            print("Regla: " , list(rules[j][0]) , " -> " , list(rules[j][1]))
            print("Soporte: " , str(soporte[j]*100))
            print("Confianza: " , str(confianza[j]*100))
            print("Sustento: " , str(sustento[j]))
            print("=====================================")


# MAIN

In [16]:
itemsets = get_frequent_itemsets(spotify, 0.020)

{'HUMBLE.': 2, 'XO TOUR Llif3': 1}
{}
[('XO TOUR Llif3', 'HUMBLE.'), ('HUMBLE.', 'Mask Off'), ('HUMBLE.', 'Congratulations')]


In [38]:
generate_association_rules(itemsets, confidence = 0, lift = 0)

1
Regla:  ['HUMBLE.']  ->  ['XO TOUR Llif3']
Soporte:  2.04
Confianza:  43.87096774193549
Sustento:  13.582342954159591
2
Regla:  ['XO TOUR Llif3']  ->  ['HUMBLE.']
Soporte:  2.04
Confianza:  63.1578947368421
Sustento:  13.582342954159591
3
Regla:  ['Mask Off']  ->  ['HUMBLE.']
Soporte:  2.04
Confianza:  64.55696202531645
Sustento:  13.883217639853001
4
Regla:  ['HUMBLE.']  ->  ['Mask Off']
Soporte:  2.04
Confianza:  43.87096774193549
Sustento:  13.883217639853001
5
Regla:  ['Congratulations']  ->  ['HUMBLE.']
Soporte:  2.1399999999999997
Confianza:  53.101736972704714
Sustento:  11.41972838122682
6
Regla:  ['HUMBLE.']  ->  ['Congratulations']
Soporte:  2.1399999999999997
Confianza:  46.02150537634409
Sustento:  11.41972838122682
